In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary 
import time

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome()
url = "https://www.everfest.com/lists/food-beer-wine-festivals" 

In [ ]:
driver.get(url)
time.sleep(10)

In [ ]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [ ]:
soup

In [ ]:
count = 0
for s in soup:
    count +=1
    print(count)
    print(s)
    break
    
# identify in inspect where this object starts and ends -- figure out if it is one review we are targeting or more
# figure out how to pick out objects using BeautifulSoup (how to do .find for div = data_react_class)

In [ ]:
results = soup.find_all("li", {"class":"page"})
print(results)

In [ ]:
element = driver.find_element_by_xpath("//*[@id='pages']/div[6]/div/div[1]/ul/li[2]/a")

In [ ]:
dir(element)

In [ ]:
element.text

In [ ]:
element.click()

In [ ]:
def ScrapeEverfest(url, browser = "chrome"):
    # create a new session
    driver = webdriver.Chrome()
    driver.get(url)

    # collecting reviews  
    more = True # flag for 'more reviews' button
    k = 0 # counting successful clicks of the 'more reviews' button
    while more:
        print('click ' + str(k))
        try: 
            # look for the button, up to 30 sec, or until the button is located
            button = button = WebDriverWait(driver, 30).until(expected_conditions.presence_of_element_located( \
                             (By.XPATH, "//a[@href=/lists/food-beer-wine-festivals/page/" + str(1+k) + "]")))            
            button.click() # try to click the button for more reviews
            k+=1 # if successful add one to the counter
        except StaleElementReferenceException as e: # if the button is not attached to the page
            print('click ' + str(k)) # print the number of successful clicks thus far
            print(e) # print the StaleElementReferenceException
            # We keep trying scene more is True
        except TimeoutException: # if the WebDriverWait times out
            # then the button did not exist with in 30 sec (NoSuchElementException)
            print('click ' + str(k)) # print the number of successful clicks thus far
            print('no more reviews') 
            more = False # the first iteration without a 'more reviews' button will be the last

    # page source formatting with Beautiful Soup
    soup = BeautifulSoup(driver.page_source, 'lxml')
    # find all reviews 
    festivalPage = soup.find_all('div',{'class':'festival-card__footer'})

    # end browser session
    driver.quit()
    print(str(len(festivalPage)) + ' festivals collected')
    return festivalPage

In [ ]:
for i in range(3):
    print('abcd%shi%sjk' % (i,i*i))

In [ ]:
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
button = WebDriverWait(driver, 30).until(expected_conditions.presence_of_element_located( \
                         (By.XPATH, "//*[@id='pages']/div[5]/div/div[1]/ul/li[5]/a")))
button.click()

In [ ]:
//*[@id="pages"]/div[5]/div/div[1]/ul/li[5]/a
//*[@id="pages"]/div[5]/div/div[1]/ul/li[8]/a
//*[@id="pages"]/div[5]/div/div[1]/ul/li[9]/a
//*[@id="pages"]/div[5]/div/div[1]/ul/li[10]/a

In [ ]:
def ProcessEverfest(festivalPage):
 
    df1 = pd.DataFrame()
    for festival in festivalPage:
        festivals = {}
        #festivals['Festival_Name'] = Festival_Name
        try:
            festivals['Festival_Name'] = festival.find(string='festival-card__title truncate gray-link').find_parent(class_=True)\
                                        .find(class_='festival-card__footer').string
        except: 
            festivals['Festival_Name'] = [None]
        try:
            festivals['Festival_Date'] = festival.find(string='festival-card__title truncate gray-link').find_parent(class_=True)\
                                        .find(class_='festival-card__date').string
        except: 
            festivals['Festival_Date'] = [None]
        try:
            festivals['Festival_Loc'] = festival.find(string='festival-card__title truncate gray-link').find_parent(class_=True)\
                                        .find(class_='festival-card__location truncate').string
        except: 
            festivals['Festival_Loc'] = [None]


        df = pd.DataFrame(festivals, index=[0])
        df1 = df1.append(df, ignore_index=True)
    return df1

In [ ]:
festivalPage = ScrapeEverfest(url = "https://www.everfest.com/lists/food-beer-wine-festivals")

In [ ]:
dat = ProcessEverfest(festivalPage = festivalPage)
dat

In [ ]:
for i in festival.find_all(class_='ReviewSection__Root-sc-189472c-0 icjcMH'):
            if 'Overall:' in i.getText().strip():
                reviews['Overall_Review'] = i.getText().replace('Overall:', '').replace('\xa0', '').strip()
            if 'Pros:' in i.getText().strip():
                reviews['Pros_Review'] = i.getText().replace('Pros:', '').replace('\xa0', '').strip()
            if 'Cons:' in i.getText().strip():
                reviews['Cons_Review'] = i.getText().replace('Cons:', '').replace('\xa0', '').strip()
            if 'Alternatives Considered:' in i.getText().strip():
                reviews['alternative'] = i.getText().replace('Alternatives Considered:', '')\
                                                    .replace('\xa0', '').strip()
            if 'Reasons for Choosing' in i.getText().strip():
                reviews['reasons_choosing'] = i.getText().replace('Reasons for Choosing ' + Company_str + ':', '')\
                                                    .replace('\xa0', '').strip()
            if 'Reasons for Switching' in i.getText().strip():
                reviews['reasons_choosing'] = i.getText().replace('Reasons for Switching to ' + Company_str + ':', '')\
                                                    .replace('\xa0', '').strip()
        for i in review.find_all(class_='ReviewSource__HalfUnitMarginDiv-lnjke6-1 hiJhEB'):
            if 'Source' not in i.getText().strip():
                reviews['Date'] = i.getText().replace('\xa0', '').strip()

In [ ]:
/html/body/div[5]/div/div[1]/ul/li[8]/a
/html/body/div[5]/div/div[1]/ul/li[9]/a
//*[@id="pages"]/div[5]/div/div[1]/ul/li[9]
//*[@id="pages"]/div[5]/div/div[1]/ul/li[6]